In [3]:
import uuid
import pandas as pd
import random
import requests
import json
import numpy as np
import cProfile
import re


from TRUCK import Truck
from API import API
from PARAMETERS import Parameters


In [4]:
def truck_dataframe(APIKEY, params):
    dict_truck = {}
    for i in range(0, len(params.list_comsumption)):
        speedprofile = random.choice(params.list_speedprofile)
        cons = params.list_comsumption[i]

        if speedprofile == "fast":
            cons = cons+0.15*cons

        create_truck = Truck("truck"+str(i), np.random.choice(params.list_energy, p=[0.85, 0.15]), cons, speedprofile,
                             random.choice(params.list_route_pm))
        new_truck = create_truck.create_truck()
        dict_truck[i] = new_truck

    df = pd.DataFrame.from_dict(dict_truck, orient='index')

    list_distance = []
    list_duration = []
    list_co2 = []

    for i in range(len(df)):
        api = API('49.1451519,1.9831777', '49.0527528,2.0388736',
                  df['vehicletype'][i], df['speedprofile'][i], df['route_pm'][i])
        json_file = api.call(APIKEY)

        distance = json_file['response']['route'][0]['summary']['distance']
        duration = json_file['response']['route'][0]['summary']['travelTime']
        co2 = json_file['response']['route'][0]['summary']['co2Emission']

        list_distance.append(distance)
        list_duration.append(duration)
        list_co2.append(co2)

    data_response = {'distance (m)': list_distance,
                     'duration (s)': list_duration,
                     'co2 (kg)': list_co2}

    df_response = pd.DataFrame(data_response)

    return df, df_response


In [5]:
APIKEY = 'nIhu0b6ZlQsJMSWcQXiZ46fXKAJmKFJgtRrUbP0ZGkw'

params = Parameters(36.2, 3, 100, ["diesel", "gasoline"], [
                    'default', 'fast'], ['fastest;truck;traffic:default'])

df, df_response = truck_dataframe(APIKEY, params)


In [ ]:
df.head()
df_response.head()